# Welcome to <font color='red'>BE</font>yond <font color='red'>T</font>witter <font color='red'>A</font>nalytics, or <font color='red'>BETA</font>:

### an app for small business and social media influencers to dig deeper into their advertisements and maximize follower-base growth

In [1]:
import glob
import os

from analysis import process_fch, process_tweets, format_ticks, analyze_data


In [2]:
available_archives = sorted(glob.glob('archives/*_tweet.json'))

print('Note, for now (until the app is public), please enter the Twitter handle and associated Tweet archive file name for one of the available archives listed here:\n')
for arc in available_archives:
    print('{}, {}\n'.format(arc.replace('archives/', '').replace('_tweet.json',''), arc.replace('archives/', '')))
    
print('\nIn the future, users will be able to upload the /data/tweet.js file from their account\'s downloaded Twitter archive.')
      

Note, for now (until the app is public), please enter the Twitter handle and associated Tweet archive file name for one of the available archives listed here:

dog_rates, dog_rates_tweet.json


In the future, users will be able to upload the /data/tweet.js file from their account's downloaded Twitter archive.


### To get started, we need to know:

In [3]:
thandle = input('What is the Twitter handle of the account you want to run BETA on?\n  ')


What is the Twitter handle of the account you want to run BETA on?
  dog_rates


In [4]:
tweet_file = input('Upload the account\'s Twitter archive:\n  ')
assert os.path.exists(os.path.join('archives', tweet_file)), 'File path does not exist. Make sure you entered the path correctly and try again.'
tweet_file = os.path.join('archives', tweet_file)


Upload the account's Twitter archive:
  dog_rates_tweet.json


### Great!

### Now let's pull your historical follower data and start searching for ads that have lost and gained you the most followers...

In [ ]:
analyze_data.get_data(thandle, tweet_file)

tweets = analyze_data.combine_archives(thandle)


Twitter handle: @dog_rates

Formatting historical follower count archive...
Done!

Formatting Tweet archive...
Done!

Combining historical follower count and Tweet archives...


### Let's extract your advertisements from the full Twitter archive.
#### Users often include #ad or #partner to distinguish advertisements from other Tweets.
#### When prompted, please identify the cluster that contains Tweets that look most like your ads (look for the hashtags and URLs that you include most frequently in your ads).

In [ ]:
ad_cluster = analyze_data.cluster_tweets(thandle, tweets)


## Here's your report!
### Each PDF provides a sample of the Tweets that lost and gained you the most followers. You can use these reports to improve your advertisements in the future!

In [ ]:
gainy_tweets, lossy_tweets = analyze_data.sig_follower_change(tweets, ad_cluster)

analyze_data.report(thandle, gainy_tweets, lossy_tweets)